In [36]:
import pandas as pd
creds = pd.read_csv("datasets/credits.csv")
print(creds.head())
print(creds.columns)


                                                cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2  [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3  [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4  [{'cast_id': 1, 'character': 'George Banks', '...   

                                                crew     id  
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    862  
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   8844  
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...  15602  
3  [{'credit_id': '52fe44779251416c91011acb', 'de...  31357  
4  [{'credit_id': '52fe44959251416c75039ed7', 'de...  11862  
Index(['cast', 'crew', 'id'], dtype='str')


In [37]:
ratings = pd.read_csv("datasets/ratings.csv")
print(ratings.head())
print(ratings.columns)
print(len(ratings))
movie_ids = []
avg_ratings = []
for movieid, group in ratings.groupby(["movieId"]):
    movie_ids.append(str(movieid[0]))
    avg_ratings.append(group["rating"].mean())

clean_ratings = pd.DataFrame({"id" : movie_ids, "avg_rating": avg_ratings})
print(clean_ratings.head())
print(len(clean_ratings))



KeyboardInterrupt: 

In [38]:
movies_meta = pd.read_csv("datasets/movies_metadata.csv")
print(movies_meta.head())
print(movies_meta.columns)
print(movies_meta["production_countries"])
print(movies_meta["release_date"])
movies_meta = movies_meta.fillna("")
print("CLEAN DF")
print(movies_meta.head())

C:\Users\Asia.LAPTOP-ASIA\AppData\Local\Temp\ipykernel_3224\1810640024.py:1: DtypeWarning: Columns (0: popularity) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_meta = pd.read_csv("datasets/movies_metadata.csv")


   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

In [43]:
import pandas as pd

# --- UNIFIED EXTRACTION: 1874 to 202 from Measuring Worth ---
print("Fetching complete historical series from MeasuringWorth...")

# We adjust the URL parameters to cover the full range
url_complete = "https://www.measuringworth.com/datasets/uscpi/result.php?year_source=1874&year_result=2020"

try:
    # 1. Read the tables from the URL
    tables = pd.read_html(url_complete)
    
    # 2. Select the data table (usually the second one)
    df = tables[1].copy()
    
    # 3. Name the columns
    df.columns = ['Year', 'CPI']
    
    # 4. Clean the data (Handle the footer notes and headers)
    # Convert to numeric, turning text notes into NaN
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df['CPI'] = pd.to_numeric(df['CPI'], errors='coerce')
    
    # Drop rows that are not numbers (like the "Average 1982-84" note)
    df = df.dropna(subset=['Year', 'CPI'])
    
    # 5. Final Formatting
    df['Year'] = df['Year'].astype(int)
    
    # 6. Calculate Purchasing Power relative to 2020
    # Formula: (CPI in 2017 / CPI in Year X)
    cpi_2020 = df.loc[df['Year'] == 2020, 'CPI'].values[0]
    df['PP_2020_Dollar'] = (cpi_2020 / df['CPI']).round(4)
    
    # 7. Save to CSV
    df.to_csv("purchasing_power_1874_2020.csv", index=False)
    
    print("Success! File saved as 'purchasing_power_1874_2020.csv'")
    print(df.tail()) # Shows the most recent years (2013-2020)

except Exception as e:
    print(f"An error occurred: {e}")


Fetching complete historical series from MeasuringWorth...
Success! File saved as 'purchasing_power_1874_2020.csv'
     Year     CPI  PP_2020_Dollar
143  2016  240.01          1.0783
144  2017  245.12          1.0559
145  2018  251.11          1.0307
146  2019  255.66          1.0123
147  2020  258.81          1.0000


In [40]:
import networkx as nx
from networkx.algorithms import bipartite
import ast
graph = nx.Graph()
creds["id"] = creds["id"].astype(str)
work_df = pd.merge(movies_meta[["title", "id", "genres", "original_language", "release_date", "production_countries", "revenue"]], creds[["cast", "id"]], on= "id", how ="inner")
#work_df = pd.merge(work_df, clean_ratings, on= "id", how ="left")
print(work_df.head())
print(work_df.columns)
print(work_df.iloc[0])
print(work_df["cast"].iloc[0])


                         title     id  \
0                    Toy Story    862   
1                      Jumanji   8844   
2             Grumpier Old Men  15602   
3            Waiting to Exhale  31357   
4  Father of the Bride Part II  11862   

                                              genres original_language  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...                en   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...                en   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...                en   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...                en   
4                     [{'id': 35, 'name': 'Comedy'}]                en   

  release_date                               production_countries  \
0   1995-10-30  [{'iso_3166_1': 'US', 'name': 'United States o...   
1   1995-12-15  [{'iso_3166_1': 'US', 'name': 'United States o...   
2   1995-12-22  [{'iso_3166_1': 'US', 'name': 'United States o...   
3   1995-12-22  [{'iso_3166_1': 

In [41]:
all_dates = []
for date in work_df["release_date"]:
    date = date[:4]
    if date != "":
        all_dates.append(int(date))

print(min(all_dates))
print(max(all_dates))


1874
2020


In [44]:
titles = work_df["title"].tolist()
genres = work_df["genres"].tolist()
languages = work_df["original_language"].tolist()
dates = work_df["release_date"].tolist()
countries = work_df["production_countries"].tolist()
cast = work_df["cast"].tolist()
#ratings = work_df["avg_rating"].tolist()
revenue = work_df["revenue"].tolist()
tupled_movie = zip(titles, genres, languages, dates, countries, revenue, cast)
edges = []
actors = []
movies = []
for el in list(tupled_movie):
    genres = ast.literal_eval(el[1])
    genres = [genre["name"] for genre in genres]
    if el[4] != "" and isinstance(el[4], str):
        try:
            countries = ast.literal_eval(el[4])
        except (ValueError, SyntaxError):
            countries = []
    else:
        countries = []
    clean_country = []
    if isinstance(countries,list) and len(countries) > 0:  
        for country in countries:
            if country["name"]:
                clean_country.append(country["name"])
    year = el[3]
    if year[:4].isdigit():
        year = int(year[:4])
        if el[-2]!= "" and not el[-2] is None and el[-2]!=[]:
            try:
                revenue = el[-2]/df[df["Year"] == year]["PP_2020_Dollar"].astype(float).values
                revenue = revenue.tolist()[0]
            except IndexError as e:
                print(el[-2])
                print(year)
    else:
        revenue = None
    movie_attributes = {"genres": genres, "countries": clean_country, "language": el[2], "revenue_2020_$" : revenue, "release_date": year}
    el_list = ast.literal_eval(el[-1])
    if len(el_list) >0:
        movie_attributes["led_by_man"] = True if el_list[0]["gender"]== 2 else False
    else:
        movie_attributes["led_by_man"] = None
    movies.append(("movie: "+ str(el[0]), movie_attributes))
    for actor in el_list:
        gender = "male" if actor["gender"] == 2 else "female"
        edge_attributes = {"character" : actor["character"]}
        if len(clean_country) > 0 and clean_country != None:
            clean_country = clean_country
        else:
            clean_country = []
        actor_attributes = {"gender": gender, "countries" : clean_country, "year":year, "genres":genres, "revenue_2020_$" : revenue}
        actors.append(("actor: "+ actor["name"], actor_attributes))
        edges.append(("movie: "+ str(el[0]), "actor: "+ actor["name"], edge_attributes))


In [ ]:
print(movies[0])

('movie: Toy Story', {'genres': ['Animation', 'Comedy', 'Family'], 'countries': ['United States of America'], 'language': 'en', 'revenue_2017_$': array([2.32251948e+08]), 'release_date': 1995, 'led_by_man': True})


In [45]:
print(len(movies))
print(len(actors)) 
print(len(edges))
print(actors[0])
# I want there to be a unique node for each actor
clean_actors_dict = {}


for name, attrs in actors:
    if name not in clean_actors_dict:
        new_attrs = attrs.copy()
        new_attrs["countries"] = set(attrs.get("countries") or [])
        genres = attrs.get("genres") or []
        genre_dict = dict()
        for genre in genres:
            genre_dict[genre] = 1
        new_attrs["genres"] = genre_dict
        year = attrs.get("year") or 0
        year_dict = dict()
        year_dict["old_hollywood"] = 0
        year_dict["new_hollywood"] = 0
        if year> 0 and year< 1967:
            year_dict["old_hollywood"] +=1
        elif year >= 1967:
            year_dict["new_hollywood"] +=1
        new_attrs["year"] = year_dict
        #new_attrs["avg_movie_rating"] = [attrs["avg_movie_rating"]]
        if not attrs["revenue_2020_$"] is None:
            new_attrs["avg_movie_revenue_2020_$"] = [int(attrs["revenue_2020_$"])]
        else:
            new_attrs["avg_movie_revenue_2020_$"] = []
        clean_actors_dict[name] = new_attrs
    else:   
        extra_countries = set(attrs.get("countries") or [])
        clean_actors_dict[name]["countries"].update(extra_countries)
        extra_genres = attrs.get("genres") or []
        for genre in extra_genres:
            if genre not in clean_actors_dict[name]["genres"]:
                clean_actors_dict[name]["genres"][genre] = 1
            else: 
                clean_actors_dict[name]["genres"][genre] +=1
        current_year = attrs["year"] or 0
        if current_year> 0 and current_year< 1967:
            clean_actors_dict[name]["year"]["old_hollywood"] +=1
        elif current_year >= 1967:
            clean_actors_dict[name]["year"]["new_hollywood"] +=1
        #clean_actors_dict[name]["avg_movie_rating"].append(attrs["avg_movie_rating"])
        if not attrs["revenue_2020_$"] is None:
            clean_actors_dict[name]["avg_movie_revenue_2020_$"].append(int(attrs["revenue_2020_$"]))

final_actors = []

for name, attrs in clean_actors_dict.items():
    attrs["countries"] = list(attrs["countries"])
    genres = list(attrs["genres"].items())
    sorted_genres = sorted(genres, key=lambda x: x[1], reverse = True)
    if len(sorted_genres)> 0:
        attrs["genre"] = sorted_genres[0][0]
    else:
        attrs["genre"] = ""
    attrs["all_genres"] = [name for name, _ in sorted_genres]
    periods = list(attrs["year"].items())
    del attrs["genres"]
    sorted_periods = sorted(periods, key=lambda x: x[1], reverse = True)
    attrs["period"] = sorted_periods[0][0]
    del attrs["year"]
    #attrs["avg_movie_rating"] = sum(attrs["avg_movie_rating"]) /len(attrs["avg_movie_rating"])
    if len(attrs["avg_movie_revenue_2020_$"]) != 0:
        attrs["avg_movie_revenue_2020_$"] = int(sum(attrs["avg_movie_revenue_2020_$"])/ len(attrs["avg_movie_revenue_2020_$"]))
    else:
        attrs["avg_movie_revenue_2020_$"] = 0
    del attrs["revenue_2020_$"]
    final_actors.append((name, attrs))

print(f"Unique actors found: {len(final_actors)}")
print(final_actors[0])



45538
563112
563112
('actor: Tom Hanks', {'gender': 'male', 'countries': ['United States of America'], 'year': 1995, 'genres': ['Animation', 'Comedy', 'Family'], 'revenue_2020_$': 219970576.49275705})
Unique actors found: 202747
('actor: Tom Hanks', {'gender': 'male', 'countries': ['Hong Kong', 'Australia', 'United Kingdom', 'Mexico', 'Hungary', 'France', 'Italy', 'United States of America', 'Singapore', 'Germany', 'India', 'Malta'], 'avg_movie_revenue_2020_$': 97984814, 'genre': 'Comedy', 'all_genres': ['Comedy', 'Drama', 'Documentary', 'Family', 'Animation', 'Thriller', 'Romance', 'History', 'Fantasy', 'Crime', 'Action', 'War', 'Adventure', 'Science Fiction', 'Horror', 'Mystery', 'Foreign', 'TV Movie'], 'period': 'new_hollywood'})


In [46]:
import networkx as nx 
graph = nx.Graph()
for movie, diz in movies:
    graph.add_node(movie, **diz, bipartite = 0)
for actor, diz in final_actors:
    if not isinstance(diz["genre"], str):
        print(diz)
    graph.add_node(actor, **diz, bipartite = 1)
for edge1, edge2, diz in edges:
    graph.add_edge(edge1, edge2, **diz)

In [47]:
movie_nodes = {n for n, d in graph.nodes(data=True) if d["bipartite"] == 0}
actor_nodes = {n for n, d in graph.nodes(data=True) if d["bipartite"] == 1}

In [51]:
import json
to_dump = nx.readwrite.json_graph.node_link_data(graph)
with open("full_graph.json", "w") as f:
    json.dump(to_dump, f)


In [52]:
# let's create a graph only for the movies produced in Hollywood (movies produced by the US, with co-production with other countries allowed)
hollywood_graph = nx.Graph()

movie_counter = 0
for movie, diz in movies:
    if "United States of America" in diz["countries"]:
        movie_counter += 1
        hollywood_graph.add_node(movie, **diz, bipartite = 0)
print(movie_counter)
actor_counter = 0
for actor, diz in final_actors:
    if "United States of America" in diz["countries"]:
        actor_counter +=1
        hollywood_graph.add_node(actor, **diz, bipartite = 1)
print(actor_counter)
edge_counter = 0
for edge1, edge2, diz in edges:
    if edge1 in hollywood_graph.nodes() and edge2 in hollywood_graph.nodes():
        edge_counter +=1
        hollywood_graph.add_edge(edge1, edge2, **diz)


21182
111458
330284


In [55]:
import json
to_dump = nx.readwrite.json_graph.node_link_data(hollywood_graph)
with open("hollywood_graph.json", "w") as f:
    json.dump(to_dump, f)
